In [ ]:
#default parameters
default_lakehouse = "wwilakehouse" 
workspace_id = "XX"
lakehouse_id = "XX"
notebook_names = "01 - Create Delta Tables,02 - Data Transformation - Business Aggregates"


In [ ]:
!pip -q install semantic-link-labs

In [ ]:
import json
import sempy_labs as labs
from sempy_labs import migration, directlake
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model
from notebookutils import notebook

In [ ]:
def update_notebooks_default_lakehouse(notebooks, new_default_lakehouse_id):
    for notebook_name in notebooks:
        # Get the current notebook definition
        notebook_def = notebookutils.notebook.getDefinition(notebook_name)
        json_payload = json.loads(notebook_def)
        
        # Check and remove any attached lakehouses
        if 'dependencies' in json_payload['metadata'] and 'lakehouse' in json_payload['metadata']['dependencies']:
        # Remove all lakehouses
            json_payload['metadata']['dependencies']['lakehouse'] = {}

            #Update new notebook definition after removing existing lakehouses and with new default lakehouseId
            (notebookutils.notebook.updateDefinition(
                        name = notebook_name,
                        content  = json.dumps(json_payload),  
                        defaultLakehouse = default_lakehouse,
                        defaultLakehouseWorkspace = workspace_id
                        )
                )
        print(f"Updated notebook {notebook_name} with new default lakehouse: {default_lakehouse} in {workspace_id}")

In [ ]:
"""
https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.resolve_lakehouse_id

Workspace is defaulted to notebook when not passed explicity and notebook is not attached to any lakehouse
"""

#default_lakehouse_id = labs.resolve_lakehouse_id(default_lakehouse)
#print(default_lakehouse_id)

#Rebind notebooks to default lakehouse of the current workspaces and remove dependecies from previous workspace
lst_Notebook = notebook_names.split(',')
update_notebooks_default_lakehouse(lst_Notebook,lakehouse_id)

print("Rebind notebooks to default lakehouse of the current workspaces and remove dependecies from previous workspace ")


In [ ]:
#Run the notebooks to rehydrate the lakehouse and load the data
for notebook_name in lst_Notebook:
    notebookutils.notebook.run(notebook_name)


In [ ]:
    labs.directlake.update_direct_lake_model_lakehouse_connection("WWI-DirectLakeModel", lakehouse = "wwilakehouse")
    #https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.update_direct_lake_model_lakehouse_connection
    
    labs.report.report_rebind("WWIReport", "WWI-DirectLakeModel")
    
    #labs.refresh_semantic_model("WWI-DirectLakeModel")
    #https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.refresh_semantic_model

In [ ]:
"""
lst_of_notebooks = notebookutils.notebook.list()

changed_notebook_id = []

for notebook_object in lst_of_notebooks:
    if (notebook_object['displayName'] in lst_Notebook ):
        changed_notebook_id.append(notebook_object['id'])

print (changed_notebook_id)            
labs.commit_to_git("Committing Notebook after rebinding process")#,"4b16dc2-4c30-4cba-9c38-c619d9acfdcc","frnuson_main_dev_workspace")

"""